# Livrable projet 3 : realisez une etude de sante publique 

In [1]:
# Import des librairies
import numpy as np
import pandas as pd

# Import des données des bilans alimentaires
veg = pd.read_csv("data/Bilan_alim_vegetal.csv")
ani = pd.read_csv("data/Bilan_alim_animal.csv")

# Ajout de la variable origin
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

# On renomme les colonnes de temp
temp.columns = ["xx","xx2","country_code","country",'xx3','element'
    ,'item_code','item','xx4',"year","unit","value",'xx5','xx6'
    ,'origin']

# # Transformation de temp en table pivot
data = temp.pivot_table(
    index=["country_code","country","item_code","item","year","origin"],
    columns = ["element"], values=["value"], aggfunc=sum)
data = data.reset_index()
data.head()

country_code  country item_code    item  year   origin  \
element                                                          
0                  1  Arménie      2511     Blé  2013  vegetal   
1                  1  Arménie      2513    Orge  2013  vegetal   
2                  1  Arménie      2514    Maïs  2013  vegetal   
3                  1  Arménie      2515  Seigle  2013  vegetal   
4                  1  Arménie      2516  Avoine  2013  vegetal   

                        value                      \
element Aliments pour animaux Autres Utilisations   
0                        93.0                 0.0   
1                       137.0                26.0   
2                        96.0                 NaN   
3                         1.0                 NaN   
4                         4.0                 NaN   

                                                        \
element Disponibilité alimentaire (Kcal/personne/jour)   
0                                               1024.0   
1                                                  0.0   
2                                                  0.0   
3                                                  1.0   
4                                                  2.0   

                                                                ...  \
element Disponibilité alimentaire en quantité (kg/personne/an)  ...   
0                                                   130.60      ...   
1                                                     0.00      ...   
2                                                     0.03      ...   
3                                                     0.12      ...   
4                                                     0.37      ...   

                                                                  \
element Disponibilité de protéines en quantité (g/personne/jour)   
0                                                    30.52         
1                                                     0.00         
2                                                     0.01         
3                                                     0.02         
4                                                     0.09         

                                                          \
element Disponibilité intérieure Exportations - Quantité   
0                          554.0                     1.0   
1                          198.0                     0.0   
2                          102.0                     NaN   
3                            1.0                     NaN   
4                            6.0                     NaN   

                                                                       \
element Importations - Quantité Nourriture Pertes Production Semences   
0                         361.0      389.0   32.0      312.0     30.0   
1                           9.0        0.0   15.0      189.0     14.0   
2                          82.0        0.0    7.0       21.0      0.0   
3                           0.0        0.0    0.0        1.0      0.0   
4                           1.0        1.0    0.0        5.0      0.0   

                                       
element Traitement Variation de stock  
0             10.0             -118.0  
1              7.0                0.0  
2              NaN                NaN  
3              NaN                0.0  
4              NaN                NaN  

[5 rows x 21 columns]

**Question 1 :**
Donnez le résultat de votre calcul pour l'année 2013, 
ainsi que pour la dernière année  disponible au jour où vous effectuez ce projet.

Dernière années disponible a ce jour : **2013**

Créez un dataframe contenant les informations de population de chaque pays. 
Calculez le nombre total d’humains sur la planète. 
Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires.

In [2]:
#Importe nos données
population_par_pays = pd.read_csv("data/Bilan_alim_Population.csv")

#Nettoye nos données
population_par_pays = population_par_pays[['Pays','Valeur']]

#On s'assure qu'il n'y est pas 2 fois le même pays
population_par_pays = population_par_pays.drop_duplicates('Pays',keep = 'first')

#On retire les information non nommer (NaN) ( Ici on retire une ligne si au moins un parametre est NaN)
population_par_pays.dropna(axis='index')

#On renome les colones
population_par_pays.columns=['country','population(1000 personnes)']



#Regarde nos valuers
population_par_pays.head()

#calcule de la popultaiton total sur terre (*1000 car l'unité precise que ces pour 1000 personne par unité de valeur)
population_total_sur_terre = population_par_pays['population(1000 personnes)'].sum() * 1000

print('La popultation humaine en 2013 etait de {} personnes'.format(population_total_sur_terre))

La popultation humaine en 2013 etait de 8413993000 personnes


Soit il y aurait eu environs 8 milliards de personnes sur terre ne 2013

Ce chiffre est un peut élevé, même si les chiffres sont arrondit, ici au millier de personnes. 
Pourtant quand on regarde au cas par cas,chaque pays semble respecter ses chiffres comme par exemple l'afghanistan,
nous avons 30 552 000 personnes et la banque modiale donne 31 700 000 personnes pour les même dates
Mais en continuant notre investigation nous remarqu'on un petit details ici :

In [3]:
population_par_pays.sort_values(by='population(1000 personnes)').tail()

,country,population(1000 personnes)
74,Indonésie,249866
53,États-Unis d'Amérique,320051
73,Inde,1252140
35,"Chine, continentale",1385567
174,Chine,1416667


Nous pouvons voire que la chine est "compter deux fois", ici la chine continantal et la chine sont compter
La chine continental represente la chine sans l'île de Taïwan et les îles voisines.
Il serait donc ici plus judicieux de faire : 

In [4]:
population_par_pays.loc[174,'population(1000 personnes)'] = population_par_pays.loc[174,'population(1000 personnes)'] - population_par_pays.loc[35,'population(1000 personnes)']

In [5]:
pop_terre = population_par_pays['population(1000 personnes)'].sum()*1000
pop_terre

7028426000

Le resultats parait déjâ un plus probable que le précédent

In [6]:
#Ajoute de la population au dataframe principale
data = pd.merge(data,population_par_pays,left_on='country', right_on='country', left_index=True, right_index=False)

#Renome les colones pour plus de claireter
data.columns = ['country','country_code','i_country','item_code','item',
               'year','origin', 'Aliments pour animaux','Autres Utilisations',
               'Disponibilité alimentaire (Kcal/personne/jour)',
               'Disponibilité alimentaire en quantité (kg/personne/an)',
               'Disponibilité de matière grasse en quantité (g/personne/jour)',
               'Disponibilité de protéines en quantité (g/personne/jour)',
               'Disponibilité intérieure','Exportations - Quantité','Importations - Quantité',
               'Nourriture','Pertes','Production','Semences','Traitement','Variation de stock','population(1000 personnes)']

#Supprime les colones inutiles
data = data.drop(columns='i_country')
data = data.reset_index()
data = data.drop(columns='index')
#On jete un petit coup d'oeil
data.head()

/home/owen/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,country,country_code,item_code,item,year,origin,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,population(1000 personnes)
0,Arménie,1,2511,Blé,2013,vegetal,93.0,0.0,1024.0,130.60,...,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0,2977
1,Arménie,1,2513,Orge,2013,vegetal,137.0,26.0,0.0,0.00,...,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0,2977
2,Arménie,1,2514,Maïs,2013,vegetal,96.0,NaN,0.0,0.03,...,102.0,NaN,82.0,0.0,7.0,21.0,0.0,NaN,NaN,2977
3,Arménie,1,2515,Seigle,2013,vegetal,1.0,NaN,1.0,0.12,...,1.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,0.0,2977
4,Arménie,1,2516,Avoine,2013,vegetal,4.0,NaN,2.0,0.37,...,6.0,NaN,1.0,1.0,0.0,5.0,0.0,NaN,NaN,2977


**Question 2 :** 

Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici). C'est une équation à 3 termes de type a1+a2+[...]=b1+b2+[...]=c1+c2+[...] ) faisant intervenir chacune des 11 quantités données ci dessus. 
Illustrez cette équation avec l'exemple du blé en France. Pour avoir un indice, cliquez sur "Définitions et Standards" sur la page de téléchargement des données.


Disponibilité interieur = (production + importation - exportation) + evoltion des stocks

Disponibilité par habitant = nourriture/ nombre d'habitant

Taux de dependance a l'importation = (importation*100)/(production +importation-exportation)

Nourriture = production + importation - (aliment pour annimaux + autre usage + perte)

Disponibilité = Disponibilié interieur + semence - perte

Disponibilité alimentaire( Kcal/personne ) = Disponibilité alimentaire (Kcal/personne/jour) * 365

Disponibilité alimentaire( Kcal ) = Disponibilité alimentaire( Kcal/personne) * populaiton

Disponibilité de matière grasse en quantité (g/personne) = Disponibilité de matière grasse en quantité (g/personne/jour)*365

Disponibilité de matière grasse en quantité (g) = Disponibilité de matière grasse en quantité (g/personne) * population

Disponibilité de protéines en quantité (g/personne) = Disponibilité de protéines en quantité (g/personne/jour)*365

Disponibilité de protéines en quantité (g) = Disponibilité de protéines en quantité (g/personne)*population




**Question 3 :**
    
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.
Vous ferez cela à partir de ces informations :

  -Population de chaque pays

  -Disponibilité alimentaire ("Food Supply" en anglais) donnée pour chaque produit et pour chaque pays en kcal/personne/jour.

  -Disponibilité alimentaire en protéines ("Protein supply quantity" en anglais) donnée pour chaque produit et pour chaque pays en g/personne/jour.


In [7]:
data['Disponibilité alimentaire (Kcal)'] = (data['Disponibilité alimentaire (Kcal/personne/jour)'] * 365) * (data['population(1000 personnes)'] * 1000)

data['Disponibilité de protéines en quantité (kg)'] = ((data['Disponibilité de protéines en quantité (g/personne/jour)'] * 365) *(data['population(1000 personnes)'] * 1000))/1000


In [8]:
#On regarde les données
data.head()

,country,country_code,item_code,item,year,origin,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,population(1000 personnes),Disponibilité alimentaire (Kcal),Disponibilité de protéines en quantité (kg)
0,Arménie,1,2511,Blé,2013,vegetal,93.0,0.0,1024.0,130.60,...,361.0,389.0,32.0,312.0,30.0,10.0,-118.0,2977,1.112684e+12,33163184.60
1,Arménie,1,2513,Orge,2013,vegetal,137.0,26.0,0.0,0.00,...,9.0,0.0,15.0,189.0,14.0,7.0,0.0,2977,0.000000e+00,0.00
2,Arménie,1,2514,Maïs,2013,vegetal,96.0,NaN,0.0,0.03,...,82.0,0.0,7.0,21.0,0.0,NaN,NaN,2977,0.000000e+00,10866.05
3,Arménie,1,2515,Seigle,2013,vegetal,1.0,NaN,1.0,0.12,...,0.0,0.0,0.0,1.0,0.0,NaN,0.0,2977,1.086605e+09,21732.10
4,Arménie,1,2516,Avoine,2013,vegetal,4.0,NaN,2.0,0.37,...,1.0,1.0,0.0,5.0,0.0,NaN,NaN,2977,2.173210e+09,97794.45


**Question 4 :**

A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire 
(pour chaque pays et chaque produit), calculez pour chaque produit le _ratio "énergie/poids"_, que vous donnerez en kcal/kg.

Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet 
par exemple en cherchant la valeur calorique d'un oeuf.

In [9]:
#calcule ratio kcal / kg

data['ratio "énergie/poids"(kcal/kg)'] = data['Disponibilité alimentaire (Kcal)'] / (data['Nourriture']*1000000)

En suivant la même méthodologie, calculez également le pourcentage de protéines de chaque produit (pour chaque pays). 
Ce pourcentage est obtenu en calculant le _ratio "poids de protéines/poids total"_ (attention aux unités utilisées). Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la teneur en protéines de l'avoine.

In [10]:
#Calcule du ratio "poids de protéines/poids total"

data['ratio "poids de protéines/poids total"'] = data['Disponibilité de protéines en quantité (kg)'] / (data['Nourriture']*1000000)

In [11]:
#On jete un coup d'oeil au données
data.head()

,country,country_code,item_code,item,year,origin,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,Pertes,Production,Semences,Traitement,Variation de stock,population(1000 personnes),Disponibilité alimentaire (Kcal),Disponibilité de protéines en quantité (kg),"ratio ""énergie/poids""(kcal/kg)","ratio ""poids de protéines/poids total"""
0,Arménie,1,2511,Blé,2013,vegetal,93.0,0.0,1024.0,130.60,...,32.0,312.0,30.0,10.0,-118.0,2977,1.112684e+12,33163184.60,2860.368946,0.085252
1,Arménie,1,2513,Orge,2013,vegetal,137.0,26.0,0.0,0.00,...,15.0,189.0,14.0,7.0,0.0,2977,0.000000e+00,0.00,NaN,NaN
2,Arménie,1,2514,Maïs,2013,vegetal,96.0,NaN,0.0,0.03,...,7.0,21.0,0.0,NaN,NaN,2977,0.000000e+00,10866.05,NaN,inf
3,Arménie,1,2515,Seigle,2013,vegetal,1.0,NaN,1.0,0.12,...,0.0,1.0,0.0,NaN,0.0,2977,1.086605e+09,21732.10,inf,inf
4,Arménie,1,2516,Avoine,2013,vegetal,4.0,NaN,2.0,0.37,...,0.0,5.0,0.0,NaN,NaN,2977,2.173210e+09,97794.45,2173.210000,0.097794


**Question 5 :**
 Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids. 
 
Étonnamment, il arrive que ce ratio soit différent en fonction du pays. 
Il faudra donc réaliser pour chaque aliment une moyenne sur les différents pays. 
Vous créerez donc une nouvelle table grâce à une agrégation. 
Attention à bien retirer les valeurs égales à 0 afin de ne pas fausser le calcul de la moyenne.

Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [12]:
#On enlevve les donner non entiere
df_normalise = pd.DataFrame(data.drop(['country_code','item_code','year','origin','Aliments pour animaux',
                                       'Autres Utilisations','Disponibilité alimentaire (Kcal/personne/jour)',
                                       'Disponibilité alimentaire en quantité (kg/personne/an)','Pertes',
                                       'Production','Semences','Traitement','Variation de stock',
                                       'population(1000 personnes)','Disponibilité alimentaire (Kcal)',
                                       'Disponibilité de protéines en quantité (kg)',
                                       'Disponibilité de matière grasse en quantité (g/personne/jour)',
                                       'Disponibilité de protéines en quantité (g/personne/jour)',
                                       'Disponibilité intérieure','Exportations - Quantité',
                                       'Importations - Quantité','Nourriture'],axis=1))

df_normalise.head()

,country,item,"ratio ""énergie/poids""(kcal/kg)","ratio ""poids de protéines/poids total"""
0,Arménie,Blé,2860.368946,0.085252
1,Arménie,Orge,NaN,NaN
2,Arménie,Maïs,NaN,inf
3,Arménie,Seigle,inf,inf
4,Arménie,Avoine,2173.210000,0.097794


In [13]:
#On remarque les valeurs 'inf' nous les remplaçons par nan car elle ne sont pas utile
pd.set_option('use_inf_as_na', True)

In [14]:
#On met en forme le dataframe en retirant les valeurs inutiles.
df_normalise = df_normalise.dropna()
df_normalise = df_normalise[df_normalise['ratio "énergie/poids"(kcal/kg)']!=0.0]

# On prepare nos donnée pour l'agregation
gb_df_normalise = df_normalise.groupby('item')

#On fait la moyenne des produits, et on renome la colone
m = gb_df_normalise.mean()
m.columns = ['ratio moyen "énergie/poids" (kcal/kg)','ratio moyen "poids de protéines/poids total"']

#On cree un dataframe avec nos donnée
moy_ratio_normalise = pd.DataFrame(m)

In [15]:
moy_ratio_normalise.sort_values('ratio moyen "énergie/poids" (kcal/kg)',ascending=False).head(20)

,"ratio moyen ""énergie/poids"" (kcal/kg)","ratio moyen ""poids de protéines/poids total"""
item,,
Huiles de Foie de Poisso,9982.932500,0.000000
Huiles de Poissons,9003.715504,0.000000
Huile de Tournesol,8832.563946,0.001321
Huil Plantes Oleif Autr,8783.921664,0.005971
Huile d'Olive,8676.641892,0.000000
Huile Graines de Coton,8675.430049,0.001975
Huile de Palme,8648.674382,0.003861
Huile de Palmistes,8602.823303,0.003047
Huile de Soja,8526.741662,0.001956


In [16]:
moy_ratio_normalise.sort_values('ratio moyen "poids de protéines/poids total"',ascending=False).head(20)

,"ratio moyen ""énergie/poids"" (kcal/kg)","ratio moyen ""poids de protéines/poids total"""
item,,
Soja,3235.286291,0.323993
Arachides Decortiquees,5662.101975,0.249889
Pois,3444.333362,0.220937
Légumineuses Autres,3391.434761,0.216494
Haricots,3369.314646,0.214103
"Plantes Oleiferes, Autre",4350.089305,0.175281
Abats Comestible,1122.440306,0.174551
"Viande, Autre",1284.403361,0.172867
Sésame,5714.990490,0.172270


On peut donc siter pour les proteines : 
    - soja
    - arachides
    - pois
    - viande,Autre
    - Viande de Bovins

On peut donc siter pour les calories : 
    - Huiles de Foie de Poisso
    - Huiles de Poissons
    - Huile de Tournesol
    - Huile d'Olive
    - Huile Graines de Coton


Donnez les résultats des questions 6 à 14 pour l'année 2013 ainsi que pour la dernière année disponible dans les données de la FAO.

Dernière année disponible = 2013

**Question 6 :**

Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [17]:
#Calcule de la disponibiliter interieur en kcal / produits
data['Disponibilité intérieure(Kcal)'] = (data['Disponibilité intérieure'] * 1000000 ) * data['ratio "énergie/poids"(kcal/kg)']

In [18]:
dispo_kcal_veg_monde = data[data['origin']=='vegetal']['Disponibilité intérieure(Kcal)'].sum()

print("Il y a {} kcal d'origine vegetal disponible dans le monde en 2013".format(int(dispo_kcal_veg_monde)))

Il y a 12372168566939328 kcal d'origine vegetal disponible dans le monde en 2013


**Question 7 :**
 
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? 
Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

On calcule la disponibiliter interieur en proteine pour les besoins de la question

In [19]:
data['Disponibilité intérieure proteine (kg)'] = data['Disponibilité intérieure'] * 1000000 * data['ratio "poids de protéines/poids total"']      

In [20]:
#Disponibiliter interieur en kg de proteine
dispo_prot_veg_monde = data[data['origin']=='vegetal']['Disponibilité intérieure proteine (kg)'].sum()

print("Il y a {} kg de proteine d'origine vegetal disponible dans le monde en 2013".format(int(dispo_prot_veg_monde)))

Il y a 291870627453 kg de proteine d'origine vegetal disponible dans le monde en 2013


Calculons ensuite combien d'humain on peut nourrire avec tout ca
Nous prendrons ces humain type: 
sources : 
http://www.fao.org/docrep/007/y5686e/y5686e08.htm

Femme de 20 a 30 ans avec un style de vie normal (activiter physique moderer) et d'un poid moyen de 55kg : 
2 450 kcal/jours soit **894 250 kcal** en 1 an

Homme de 20 a 30 ans avec un style de vie normal (activiter physique moderer) et d'un poid moyen de 75kg : 
3 450 kcal/jours soit **1 259 250 kcal** en 1 an

**Un humain lambda : 2 500 kcal/jours soit __912 500kcal__ en 1 an




Pour les proteine nous garderons nos humains type si dessus:
sources : 
https://www.cerin.org/actualites/besoin-en-proteines-des-sportifs-aspects-quantitatif-et-qualitatif/
        
Femme style de vie normal (sedentaire) 55kg : 0,83g/kg/j soit 45.65 g/j ou **16662.25 g** de proteine en 1 an (16.66225Kg)

Homme style de vie normal (sedentaire) 75kg : 0,83g/kg/j soit 62.25 g/j ou **22721.25 g** de proteine en 1 an (22.72125Kg)

Soit notre humain lambda a **19691.75 g** en 1 an (19.69175 kg)




In [21]:
#On arroundi un peut nos chiffres
humain_kcal = 912500
humain_prot = 20


nb_perosnne_kcal = int(dispo_kcal_veg_monde/humain_kcal)
nb_perosnne_prot = int(dispo_prot_veg_monde/humain_prot)

In [22]:
print("On pourrait donc nourrire {} personnes avec les kcal et {} avec les proteine".format(nb_perosnne_kcal,nb_perosnne_prot))

On pourrait donc nourrire 13558540895 personnes avec les kcal et 14593531372 avec les proteine


On passe le tout en pourcentage

In [23]:
porcent_pop_kcal = int(nb_perosnne_kcal * 100 / pop_terre)
pourcent_pop_prot = int(nb_perosnne_prot * 100 / pop_terre)

In [24]:
print('{} % de la population de kcal et {} % de pop en prot'.format(porcent_pop_kcal,pourcent_pop_prot))

192 % de la population de kcal et 207 % de pop en prot


**Question 8 :** 
    
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture ? 

Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

Disponibiliter mondiale en kcal vegetal en prennant les pertes et les aliments pour animaux

In [25]:
dispo_kcal_veg_monde_total = (data[data['origin']=='vegetal']['Nourriture']+data[data['origin']=='vegetal']['Pertes']+data[data['origin']=='vegetal']['Aliments pour animaux']) * 1000000 * data[data['origin']=='vegetal']['ratio "énergie/poids"(kcal/kg)']

In [26]:
dispo_kcal_veg_monde_total_sum = dispo_kcal_veg_monde_total.sum()
print('Il y a {} kcal vegetal'.format(dispo_kcal_veg_monde_total_sum))

Il y a 6142529351578714.0 kcal vegetal


On fait de même pour les proteines

In [27]:
dispo_prot_veg_monde_total = (data[data['origin']=='vegetal']['Nourriture'] + data[data['origin']=='vegetal']['Pertes'] + data[data['origin']=='vegetal']['Aliments pour animaux']) * 1000000 * data[data['origin']=='vegetal']['ratio "poids de protéines/poids total"']

In [28]:
dispo_prot_veg_monde_total_sum = dispo_prot_veg_monde_total.sum()
print('Il y a {} prot vegetal'.format(dispo_prot_veg_monde_total_sum))

Il y a 161474452347.68753 prot vegetal


On calcule le nombres d'humains nourrisable avec 
en gardant le même _humain type_

In [29]:
nb_perosnne_kcal_tt = int(dispo_kcal_veg_monde_total_sum/humain_kcal)
nb_perosnne_prot_tt = int(dispo_prot_veg_monde_total_sum/humain_prot)

#En pourcentage
pourcent_pop_kcal_tt = int(nb_perosnne_kcal_tt * 100 / pop_terre)
pourcent_pop_prot_tt = int(nb_perosnne_prot_tt * 100 / pop_terre)

In [30]:
print('{} % de la population de kcal et {} % de pop en prot'.format(pourcent_pop_kcal_tt,pourcent_pop_prot_tt))

95 % de la population de kcal et 114 % de pop en prot


**Voire les resulat pour les prots**

**Question 9 :** 

Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

On reprendre les meme calcule que la question précédente sans filter uniquement les vegetaux .

In [31]:
dispo_kcal_monde_total = data['Nourriture'] * data['ratio "énergie/poids"(kcal/kg)']
dispo_kcal_monde_total_sum = dispo_kcal_monde_total.sum()

dispo_prot_monde_total = data['Nourriture'] * 1000000 * data['ratio "poids de protéines/poids total"']
dispo_prot_monde_total_sum = dispo_prot_monde_total.sum()

In [32]:
nb_perosnne_kcal_monde = int(dispo_kcal_monde_total_sum/humain_kcal)
nb_perosnne_prot_monde = int(dispo_prot_monde_total_sum/humain_prot)

#En pourcentage
pourcent_pop_kcal_monde = int(dispo_kcal_monde_total_sum * 100 / pop_terre)
pourcent_pop_prot_monde = int(dispo_prot_monde_total_sum * 100 / pop_terre)

In [33]:
print('{} % de la population de kcal et {} % de pop en prot'.format(pourcent_pop_kcal_monde,pourcent_pop_prot_monde))

105 % de la population de kcal et 2963 % de pop en prot


 **Question 10 :**
        
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

Preparont un dataframe des données de sous nutrission.


In [34]:
#importation
data_secu = pd.read_csv('data/DonneeSec_Personne_sousAlimenter.csv')

#On modifie la chine comme fait pour la question 1
data_secu.loc[data_secu['Zone'] == 'Chine','Valeur'] = data_secu[data_secu['Zone'] == 'Chine']['Valeur'] - data_secu[data_secu['Zone'] == 'Chine, continentale']['Valeur']

#Regarde la forme des données
data_secu.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,8.1,F,Estimation FAO
1,FS,Données de la sécurité alimentaire,202,Afrique du Sud,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,2.6,F,Estimation FAO
2,FS,Données de la sécurité alimentaire,3,Albanie,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,0.2,F,Estimation FAO
3,FS,Données de la sécurité alimentaire,4,Algérie,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,1.8,F,Estimation FAO
4,FS,Données de la sécurité alimentaire,79,Allemagne,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,NaN,<2,Moins de 2 millions de personnes sont sous-ali...


Les colones sont assez explixite donc il n'y a pas besoin de les renommées

La question est la suivante : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?
    Pour ce faire nous allons simplement additionner toute les personnes sous alimenter presente dans ce tableau : "Nombre de personnes sous-alimentées (millions)"
     et en faire un pourcentage de la population mondiale total que nous avons trouver dans les questions precedentes

In [35]:
nb_personne_sous_alim = data_secu[data_secu["Produit"]=="Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)"]['Valeur'].sum()

print("soit il y aurait {} millions de personnes soufrant de sous alimentation dans le monde".format(nb_personne_sous_alim))

soit il y aurait 739.7 millions de personnes soufrant de sous alimentation dans le monde


In [36]:
#On ramene notre population en uniter et on la passe en % de la population du monde.
nb_personne_sous_alim_pourcent  = nb_personne_sous_alim * 1000000 * 100 / pop_terre

print('Ce qui represente donc {} % de la population'.format(int(nb_personne_sous_alim_pourcent)))

Ce qui represente donc 10 % de la population


Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.

Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal").

Nous allons donc ajouter une colone a notre dataframe _data_ nommer "is_cereal" qui sera un booléen, il sera _True_ si le produit est un céréale.

In [37]:
# Import des données des céréales seulement
temp_data_cereal = pd.read_csv('data/Bilan_alim_cerealBiere.csv')

#Petite table pivot rapide pour recupere la meme mise en forme que le dataframe principale
temp_data_cereal = temp_data_cereal.pivot_table(
    index=["Code Pays","Pays","Code Produit","Produit","Année"],
    columns = ['Élément'], values=["Valeur"], aggfunc=sum)

temp_data_cereal = temp_data_cereal.reset_index()

#Isole les colones qu'on veut
a = pd.unique(temp_data_cereal['Code Produit'])
b = pd.unique(temp_data_cereal['Produit'])

#Crée un dataframe avec les codes et les produit
df_liste_cereal = pd.DataFrame({'item_code':a,'item':b})

Nous avons donc la liste des produits considere comme céréale par le fao 

In [38]:
df_liste_cereal

,item_code,item
0,2511,Blé
1,2513,Orge
2,2514,Maïs
3,2515,Seigle
4,2516,Avoine
5,2517,Millet
6,2518,Sorgho
7,2520,"Céréales, Autres"
8,2805,Riz (Eq Blanchi)


Il nous faut mainteant crée notre colone sur le dataframe principale et la remplire correctement

In [39]:
#On separe pour plus de claireter
f = data['item_code'].isin(df_liste_cereal['item_code'])

#On remplis la colones avec les valeur qui sont connu
data.loc[f,'is_cereal']=True

#Et on mes les false a la place des nan qui arrive par defaut
data['is_cereal'] = data['is_cereal'].fillna(False)

In [40]:
#On jete un coup d'oeil
data[['country','item','is_cereal']].head(10)

,country,item,is_cereal
0,Arménie,Blé,True
1,Arménie,Orge,True
2,Arménie,Maïs,True
3,Arménie,Seigle,True
4,Arménie,Avoine,True
5,Arménie,Millet,True
6,Arménie,Sorgho,True
7,Arménie,"Céréales, Autres",True
8,Arménie,Pommes de Terre,False
9,Arménie,Manioc,False


**Question 11 :**

En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [41]:
#Calcule le nb de kg de céréale pour animaux
cereal_animau_total_pds = data[data['is_cereal']]['Aliments pour animaux'].sum() * 1000000

#Calcule le nb de kg de céréale pour alimentation ( animaux et humains)
cereal_alimentation = data[data['is_cereal']]['Aliments pour animaux'] + data[data['is_cereal']]['Nourriture']
cereal_alimentation_total_pds = cereal_alimentation.sum() *1000000

#On fait un pourcentage du poids des cereale pour animaux en fonction du poids total
taux_cereal_animal = cereal_animau_total_pds * 100 / cereal_alimentation_total_pds
print("Ainsi les animaux on {} % de la totaliter des céréales a destination de l'alimentation".format(int(taux_cereal_animal)))

Ainsi les animaux on 48 % de la totaliter des céréales a destination de l'alimentation


**Preparation suite**

Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition, pour une année choisie.(pourcentage par pays)

Repérez les 15 produits les plus exportés par ce groupe de pays sur l'année choisie.

Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné sur l'année choisie)

Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.

le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [42]:
data = pd.merge(data,data_secu,left_on='country',right_on='Zone',how='left')
data = data.drop(['Code Domaine','Domaine', 'Code zone', 'Zone', 
                          'Code Élément', 'Élément','Code Produit', 'Produit', 
                          'Code année', 'Année', 'Unité','Symbole', 'Description du Symbole'],axis=1)

In [43]:
data = data.rename(index=str,columns={'Valeur':'Nombre de personnes sous-alimentées (millions)'})
data['Pourcent population sous-alimnetées'] =( data['Nombre de personnes sous-alimentées (millions)'] * 1000000) * 100 / (data['population(1000 personnes)']*1000)
data.head()

,country,country_code,item_code,item,year,origin,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,population(1000 personnes),Disponibilité alimentaire (Kcal),Disponibilité de protéines en quantité (kg),"ratio ""énergie/poids""(kcal/kg)","ratio ""poids de protéines/poids total""",Disponibilité intérieure(Kcal),Disponibilité intérieure proteine (kg),is_cereal,Nombre de personnes sous-alimentées (millions),Pourcent population sous-alimnetées
0,Arménie,1,2511,Blé,2013,vegetal,93.0,0.0,1024.0,130.60,...,2977,1.112684e+12,33163184.60,2860.368946,0.085252,1.584644e+12,4.722983e+07,True,0.1,3.359086
1,Arménie,1,2513,Orge,2013,vegetal,137.0,26.0,0.0,0.00,...,2977,0.000000e+00,0.00,NaN,NaN,NaN,NaN,True,0.1,3.359086
2,Arménie,1,2514,Maïs,2013,vegetal,96.0,NaN,0.0,0.03,...,2977,0.000000e+00,10866.05,NaN,NaN,NaN,NaN,True,0.1,3.359086
3,Arménie,1,2515,Seigle,2013,vegetal,1.0,NaN,1.0,0.12,...,2977,1.086605e+09,21732.10,NaN,NaN,NaN,NaN,True,0.1,3.359086
4,Arménie,1,2516,Avoine,2013,vegetal,4.0,NaN,2.0,0.37,...,2977,2.173210e+09,97794.45,2173.210000,0.097794,1.303926e+10,5.867667e+05,True,0.1,3.359086


On a maintenant le pourcentage de personne sous alimenter par pays.

In [44]:
#On selectionne tout les pays qui sont present dans la liste des pays avec de la sous alimentation
data_secu_sousalim = data[data['Pourcent population sous-alimnetées']>1]

In [45]:
#On trie les valeurs par ordre decroissant de quantiter d'exportation
produit_plus_export = data_secu_sousalim.sort_values(['Exportations - Quantité'],ascending=False)

#On affiche les 15 premier produits
produit_plus_export[['country','item','Exportations - Quantité']].head(15)

,country,item,Exportations - Quantité
13344,Thaïlande,Manioc,25214.0
6165,Indonésie,Huile de Palme,23333.0
508,Argentine,Maïs,20241.0
8455,Malaisie,Huile de Palme,19622.0
6125,Inde,Riz (Eq Blanchi),11337.0
14724,Viet Nam,Manioc,8973.0
2534,"Chine, continentale","Légumes, Autres",8084.0
525,Argentine,Soja,7783.0
6031,Inde,Blé,7168.0
14796,Viet Nam,Riz (Eq Blanchi),6951.0


Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné sur l'année choisie)

 Ici on doit trouver les 200 plus grande importation des produit que l'ont a trouver juste au dessus

In [46]:
#On isole les produits les plus exporter
liste_produit_plus_export = produit_plus_export['item'].head(15).unique()

In [47]:
#On prend les donner de data seulement pour les produits les plus exporter
produit_les_plus_import = data[data['item'].isin(liste_produit_plus_export)]

#On trie ces produits par quantiter d'import
produit_les_plus_import = produit_les_plus_import.sort_values(['Importations - Quantité'],ascending=False)

#On crée une variable pour les 200 plus gros import 
deux_cent_plus_gros_import = produit_les_plus_import[['country','item','Importations - Quantité']].head(200)
deux_cent_plus_gros_import.head(15)

,country,item,Importations - Quantité
15627,Chine,Soja,65564.0
2509,"Chine, continentale",Soja,63381.0
15614,Chine,Manioc,30466.0
2496,"Chine, continentale",Manioc,29046.0
6965,Japon,Maïs,14403.0
3674,Égypte,Blé,10331.0
7490,République de Corée,Maïs,8755.0
6069,Inde,Huile de Palme,8568.0
1333,Brésil,Blé,7630.0
15605,Chine,Blé,7572.0


In [48]:
#On a tout nos produit trier par groupe de produit
produit_les_plus_import.sort_values(['item'],ascending=False).groupby('item').head()

,country,country_code,item_code,item,year,origin,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,population(1000 personnes),Disponibilité alimentaire (Kcal),Disponibilité de protéines en quantité (kg),"ratio ""énergie/poids""(kcal/kg)","ratio ""poids de protéines/poids total""",Disponibilité intérieure(Kcal),Disponibilité intérieure proteine (kg),is_cereal,Nombre de personnes sous-alimentées (millions),Pourcent population sous-alimnetées
801,Bahamas,12,2542,Sucre Eq Brut,2013,vegetal,NaN,0.0,305.0,29.58,...,377,4.196952e+10,NaN,3.815411e+03,NaN,4.196952e+10,NaN,False,NaN,NaN
5134,Grenade,86,2542,Sucre Eq Brut,2013,vegetal,NaN,0.0,355.0,36.18,...,106,1.373495e+10,NaN,3.433738e+03,NaN,1.373495e+10,NaN,False,NaN,NaN
1528,Îles Salomon,25,2542,Sucre Eq Brut,2013,vegetal,NaN,NaN,41.0,4.05,...,561,8.395365e+09,NaN,4.197682e+03,NaN,8.395365e+09,NaN,False,NaN,NaN
1074,Bermudes,17,2542,Sucre Eq Brut,2013,vegetal,NaN,0.0,235.0,22.96,...,65,5.575375e+09,NaN,5.575375e+03,NaN,1.115075e+10,NaN,False,NaN,NaN
2784,Costa Rica,48,2542,Sucre Eq Brut,2013,vegetal,NaN,0.0,467.0,48.00,...,4872,8.304568e+11,NaN,3.548961e+03,NaN,8.304568e+11,NaN,False,0.3,6.157635
620,Australie,10,2555,Soja,2013,vegetal,3.0,NaN,1.0,0.19,...,23343,8.520195e+09,7.668176e+05,2.130049e+03,0.191704,1.917044e+11,1.725339e+07,False,NaN,NaN
10636,Tchéquie (la),167,2555,Soja,2013,vegetal,NaN,NaN,7.0,0.82,...,10702,2.734361e+10,3.320296e+06,3.038179e+03,0.368922,1.458326e+11,1.770824e+07,False,NaN,NaN
11011,Pologne,173,2555,Soja,2013,vegetal,NaN,0.0,1.0,0.06,...,38217,1.394920e+10,8.369523e+05,6.974602e+03,0.418476,3.766285e+11,2.259771e+07,False,NaN,NaN
3068,Azerbaïdjan,52,2555,Soja,2013,vegetal,11.0,NaN,0.0,0.00,...,9413,0.000000e+00,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
3251,Danemark,54,2555,Soja,2013,vegetal,13.0,NaN,0.0,0.03,...,5619,0.000000e+00,2.050935e+04,NaN,NaN,NaN,NaN,False,NaN,NaN


Calcule des ratio suivant:

le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.

le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [49]:
df_export = pd.DataFrame(produit_les_plus_import.sort_values(['item'],ascending=False))
d = pd.pivot_table(produit_les_plus_import,index='item')
d = d.drop(['Disponibilité alimentaire (Kcal)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de protéines en quantité (kg)',
        'Disponibilité intérieure proteine (kg)',
       'Disponibilité intérieure(Kcal)', 'Exportations - Quantité',
       'Importations - Quantité', 'Pertes', 'Production',
       'Semences', 'Traitement', 'Variation de stock', 'country_code',
       'is_cereal', 'item_code', 'population(1000 personnes)',
       'ratio "poids de protéines/poids total"',
       'ratio "énergie/poids"(kcal/kg)', 'year'],axis=1)

In [50]:
d['ratio"Autres utilisations" (Other uses)/disponibilité intérieure'] = d['Autres Utilisations']/d['Disponibilité intérieure']
d['ratio  nourriture animale/quantité nourriture(animale + humaine)'] = d['Aliments pour animaux']/d['Nourriture']

In [51]:
d.head()

,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Nombre de personnes sous-alimentées (millions),Nourriture,Pourcent population sous-alimnetées,"ratio""Autres utilisations"" (Other uses)/disponibilité intérieure",ratio nourriture animale/quantité nourriture(animale + humaine)
item,,,,,,,,
Bananes,45.842105,6.545455,691.875740,7.786813,573.476190,16.409968,0.009460,0.079937
Blé,1563.620000,187.940741,4608.308571,7.705208,3126.914286,16.541091,0.040783,0.500052
Huile de Palme,0.000000,233.252941,337.222222,7.853191,144.233871,16.550582,0.691689,0.000000
"Légumes, Autres",999.388235,13.756757,7915.942857,7.705208,6812.434286,16.541091,0.001738,0.146701
Manioc,1088.405941,386.700855,1735.648810,7.711111,819.793651,16.433302,0.222799,1.327658


**Question 12 :**
    
Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

In [52]:
d.sort_values('ratio"Autres utilisations" (Other uses)/disponibilité intérieure',ascending=False).head(3)

,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Nombre de personnes sous-alimentées (millions),Nourriture,Pourcent population sous-alimnetées,"ratio""Autres utilisations"" (Other uses)/disponibilité intérieure",ratio nourriture animale/quantité nourriture(animale + humaine)
item,,,,,,,,
Huile de Palme,0.000000,233.252941,337.222222,7.853191,144.233871,16.550582,0.691689,0.000000
Maïs,4267.792683,1727.607692,6703.205714,7.705208,788.315789,16.541091,0.257729,5.413811
Manioc,1088.405941,386.700855,1735.648810,7.711111,819.793651,16.433302,0.222799,1.327658


In [53]:
d.sort_values('ratio  nourriture animale/quantité nourriture(animale + humaine)',ascending=False).head(3)

,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Nombre de personnes sous-alimentées (millions),Nourriture,Pourcent population sous-alimnetées,"ratio""Autres utilisations"" (Other uses)/disponibilité intérieure",ratio nourriture animale/quantité nourriture(animale + humaine)
item,,,,,,,,
Maïs,4267.792683,1727.607692,6703.205714,7.705208,788.315789,16.541091,0.257729,5.413811
Soja,428.666667,58.090909,2065.922156,8.128090,99.611111,15.855712,0.028119,4.303402
Poissons Pelagiques,182.000000,2.390805,294.166667,7.701053,127.775862,16.436132,0.008127,1.424369


**Question 13 :**

Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [54]:
test1 = data[data['country']=="États-Unis d'Amérique"]

test1 = test1[test1['is_cereal']]

k = test1['Aliments pour animaux'].sum()

In [55]:
j = k * 10 / 100
print("On pourrait libéré {} tonnes de cereals".format(j))

On pourrait libéré 14009.6 tonnes de cereals


**Question 14 :**
    
En Thaïlande, quel proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [56]:
manioc_thai = data[(data['country']=='Thaïlande') & (data['item']=='Manioc')]
manioc_thai_export = manioc_thai['Exportations - Quantité']
manioc_thai_total = manioc_thai['Production']

In [57]:
proportion_export = manioc_thai_export * 100 / manioc_thai_total

In [58]:
print("La Thaïlande exporte {} % de ca production de manioc".format(int(proportion_export.values)))

La Thaïlande exporte 83 % de ca production de manioc


In [59]:
p = data[data['country']=='Thaïlande']['Pourcent population sous-alimnetées'].unique()
print("La Thaïlande a {} % de personne en sous nutrission".format(int(p)))

La Thaïlande a 9 % de personne en sous nutrission


La partie calculatoire etant fini, nous allons exporter nos données.Mais il nous faut deja les preparer a l'export. Nous allons donc crée 4 fichier d'export.

In [60]:
#Table population
df_pop = pd.DataFrame(data[['country', 'country_code','year','population(1000 personnes)']])
df_pop = df_pop.drop_duplicates()

#Table disponibiliter alimentaire
df_dispo = pd.DataFrame(data[['country', 'country_code','year', 'item', 'item_code', 'origin','Disponibilité intérieure','Disponibilité alimentaire (Kcal/personne/jour)','Disponibilité de matière grasse en quantité (g/personne/jour)','Disponibilité de matière grasse en quantité (g/personne/jour)']])

#Table equilibre de production
df_prod = pd.DataFrame(data[['country', 'country_code','year', 'item', 'item_code','Disponibilité intérieure','Aliments pour animaux','Semences','Pertes','Traitement','Nourriture','Autres Utilisations']])

#Table personnes sous alimenter
df_sousali = pd.DataFrame(data[['country', 'country_code','year','Nombre de personnes sous-alimentées (millions)']])
df_sousali = df_sousali.drop_duplicates()


In [61]:
df_pop.to_csv("data/csv_out/pop.csv", index = False)
df_dispo.to_csv("data/csv_out/dispo.csv", index = False)
df_prod.to_csv("data/csv_out/prod.csv", index = False)
df_sousali.to_csv("data/csv_out/sousalim.csv", index = False)
